In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn import datasets

In [3]:
data = datasets.load_digits()

In [4]:
X = data.data

In [5]:
X[ :1, :]

array([[  0.,   0.,   5.,  13.,   9.,   1.,   0.,   0.,   0.,   0.,  13.,
         15.,  10.,  15.,   5.,   0.,   0.,   3.,  15.,   2.,   0.,  11.,
          8.,   0.,   0.,   4.,  12.,   0.,   0.,   8.,   8.,   0.,   0.,
          5.,   8.,   0.,   0.,   9.,   8.,   0.,   0.,   4.,  11.,   0.,
          1.,  12.,   7.,   0.,   0.,   2.,  14.,   5.,  10.,  12.,   0.,
          0.,   0.,   0.,   6.,  13.,  10.,   0.,   0.,   0.]])

In [6]:
y = data.target
y[:5]

array([0, 1, 2, 3, 4])

### Начнем с логистической регрессии LogisticRegression

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import StratifiedShuffleSplit

D:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
sss = StratifiedShuffleSplit(y)

In [9]:
#Создадим лист куда будем записывать модель и полученную точность
models_accuracy = {}

In [10]:
accuracies = []
lr = LogisticRegression(multi_class='multinomial',  penalty='l2', solver='lbfgs')
C_s = np.linspace(0.1, 2, 10)
for C in C_s:
    lr.C = C
    accuracies.append(np.mean(cross_val_score(lr, X, y, cv=sss, scoring='accuracy')))

In [11]:
for C, accuracy in zip(C_s, accuracies):
    print("Значение С = {}, точность = {}".format(round(C,2), round(accuracy, 3)))

Значение С = 0.1, точность = 0.961
Значение С = 0.31, точность = 0.966
Значение С = 0.52, точность = 0.969
Значение С = 0.73, точность = 0.966
Значение С = 0.94, точность = 0.966
Значение С = 1.16, точность = 0.961
Значение С = 1.37, точность = 0.961
Значение С = 1.58, точность = 0.967
Значение С = 1.79, точность = 0.966
Значение С = 2.0, точность = 0.965


In [12]:
lr.C = C_s[accuracies.index(max(accuracies))]

In [13]:
models_accuracy[lr] = max(accuracies)

Запишем нашу первую полученную модель и ее точность.

In [14]:
models_accuracy

{LogisticRegression(C=0.52222222222222225, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='multinomial', n_jobs=1, penalty='l2',
           random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
           warm_start=False): 0.9688888888888888}

### Переходим к рассмотрению решающего дерева DecisionTreeClassifier (код с занятия).

In [15]:
from sklearn.tree import DecisionTreeClassifier

In [16]:
means = []
depths = list(range(2, 20))
for depth in depths:
    clf = DecisionTreeClassifier(max_depth=depth)
    means.append(cross_val_score(estimator=clf, X=X, y=y, scoring='accuracy', cv=sss).mean())

In [17]:
for depth, mean in zip(depths, means):
    print("Глубина дерева = {}, точность = {}".format(round(depth,2), round(mean, 3)))

Глубина дерева = 2, точность = 0.326
Глубина дерева = 3, точность = 0.467
Глубина дерева = 4, точность = 0.563
Глубина дерева = 5, точность = 0.669
Глубина дерева = 6, точность = 0.743
Глубина дерева = 7, точность = 0.818
Глубина дерева = 8, точность = 0.838
Глубина дерева = 9, точность = 0.849
Глубина дерева = 10, точность = 0.852
Глубина дерева = 11, точность = 0.851
Глубина дерева = 12, точность = 0.848
Глубина дерева = 13, точность = 0.871
Глубина дерева = 14, точность = 0.856
Глубина дерева = 15, точность = 0.853
Глубина дерева = 16, точность = 0.859
Глубина дерева = 17, точность = 0.873
Глубина дерева = 18, точность = 0.848
Глубина дерева = 19, точность = 0.854


Наилучшие результаты показали деревья глубиной от 9 до 19. Самую большую точность имеет дерево глубиной - 12.

In [18]:
clf.max_depth = depths[means.index(max(means))]

In [20]:
models_accuracy[clf] = max(means)

In [21]:
models_accuracy

{LogisticRegression(C=0.52222222222222225, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='multinomial', n_jobs=1, penalty='l2',
           random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
           warm_start=False): 0.9688888888888888,
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=17,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
             splitter='best'): 0.87277777777777776}

## Рассмотрим следующий алгоритм SVM.
Рассмотрим различные параметры C. Параметр С позволяет находить компромисс между максимизацией разделяющей полосы и минимизацией суммарной ошибки.

In [22]:
from sklearn import svm

In [23]:
svc = svm.SVC(kernel='linear')

In [24]:
C_s = np.logspace(-10, 0, 25)

In [25]:
import matplotlib.pyplot as plt

In [26]:
means = []
for C in C_s:
    svc.C = C
    means.append(np.mean(cross_val_score(svc, X, y, scoring='accuracy', cv=sss)))

In [27]:
for c, mean in zip(C_s, means):
    print("Параметр штрафа С = {}, точность = {}".format(round(c, 2), round(mean, 3)))

Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.194
Параметр штрафа С = 0.0, точность = 0.812
Параметр штрафа С = 0.0, точность = 0.933
Параметр штрафа С = 0.0, точность = 0.957
Параметр штрафа С = 0.0, точность = 0.974
Параметр штрафа С = 0.0, точность = 0.982
Параметр штрафа С = 0.0, точность = 0.982
Параметр штрафа С = 0.0, точность = 0.979
Параметр штрафа С = 0.01, точность = 0.981
Параметр штрафа С = 0.02, точность = 0.978
Параметр штрафа С = 0.06, точность = 0.983
Параметр штрафа С = 0.15, точность = 0.979
Параметр штрафа С = 0.38, точность = 0.978
Параметр 

In [28]:
svc.C = C_s[means.index(max(means))]

In [30]:
models_accuracy[svc] = max(means)

In [31]:
models_accuracy

{SVC(C=0.056234132519034911, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False): 0.98277777777777775,
 LogisticRegression(C=0.52222222222222225, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='multinomial', n_jobs=1, penalty='l2',
           random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
           warm_start=False): 0.9688888888888888,
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=17,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
             splitter='best'): 0.87277777777777776}

### Рассмотрим метод KNN (метод ближайших соседей).

In [32]:
from sklearn.neighbors import KNeighborsClassifier

In [33]:
knn = KNeighborsClassifier()
weights = ['uniform', 'distance']

In [34]:
means_uniform = []
#Рассмотрим сначала модель, где от расстояния не зависит вес
knn.weights = 'uniform'
for n_neighbors in list(range(2, 10)):
    knn.n_neighbors = n_neighbors
    means_uniform.append(np.mean(cross_val_score(knn, X, y, scoring='accuracy', cv=sss)))

In [35]:
means_uniform

[0.98777777777777787,
 0.98944444444444435,
 0.98888888888888893,
 0.99111111111111116,
 0.9850000000000001,
 0.98388888888888881,
 0.98277777777777775,
 0.97888888888888881]

In [36]:
means_distance = []
#Рассмотрим сначала модель, где от расстояния не зависит вес
knn.weights = 'distance'
for n_neighbors in list(range(2, 10)):
    knn.n_neighbors = n_neighbors
    means_distance.append(np.mean(cross_val_score(knn, X, y, scoring='accuracy', cv=sss)))

In [37]:
means_distance

[0.98833333333333351,
 0.99222222222222223,
 0.98333333333333339,
 0.98944444444444457,
 0.98166666666666669,
 0.99222222222222212,
 0.98777777777777787,
 0.98555555555555563]

In [38]:
max(means_uniform) > max(means_distance) #чуть лучше оказалась модель, где учитывется расстоние

False

In [39]:
knn.n_neighbors = list(range(2, 10))[means_distance.index(max(means_distance))]
knn.weights = 'distance'

In [40]:
models_accuracy[knn] = max(means_distance)

In [41]:
models_accuracy

{SVC(C=0.056234132519034911, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False): 0.98277777777777775,
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
            metric_params=None, n_jobs=1, n_neighbors=3, p=2,
            weights='distance'): 0.99222222222222223,
 LogisticRegression(C=0.52222222222222225, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='multinomial', n_jobs=1, penalty='l2',
           random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
           warm_start=False): 0.9688888888888888,
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=17,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_

## Решающие деревья показали очень низкую точнсть, попробуем использовать ансамбли.

In [42]:
from sklearn.ensemble import RandomForestClassifier

In [43]:
rfc = RandomForestClassifier()

In [44]:
amoun_estimators = list(range(2, 100, 5))

In [45]:
means = []
for n_estimators in amoun_estimators:
    rfc.n_estimators = n_estimators
    means.append(np.mean(cross_val_score(rfc, X, y, scoring='accuracy', cv=sss)))

In [46]:
means

[0.78277777777777779,
 0.93777777777777782,
 0.96055555555555561,
 0.9655555555555555,
 0.96444444444444444,
 0.97166666666666646,
 0.97111111111111104,
 0.98111111111111104,
 0.9722222222222221,
 0.97777777777777786,
 0.97666666666666657,
 0.97277777777777774,
 0.97944444444444445,
 0.97944444444444445,
 0.97833333333333328,
 0.97499999999999998,
 0.97944444444444445,
 0.97722222222222221,
 0.97888888888888892,
 0.97166666666666646]

In [47]:
rfc.n_estimators = amoun_estimators[means.index(max(means))]

In [48]:
models_accuracy[rfc] = max(means)

In [49]:
models_accuracy

{SVC(C=0.056234132519034911, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False): 0.98277777777777775,
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
            metric_params=None, n_jobs=1, n_neighbors=3, p=2,
            weights='distance'): 0.99222222222222223,
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=37, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False): 0.98111111111111104,
 LogisticRegression(C=0.52222222222222225, class_weight=None, dual=False,
           fit_

## Итоги. 
Самая худшая модель - DecisionTreeClassifier с точностью  = __0.873__, самая лучшая модель - KNeighborsClassifier с точностью = __0.992__. 
В KNeighborsClassifier используем следующие параметры: 
* количество соседей (__n_neighbors__) = 3
* веса (__weights__) = 'distance', то есть дальность напрямую влияет на расчет в модели